# Scraping PCPartPicker

### 👨‍💻 Autores del proyecto

* [Alejandro Barrionuevo Rosado](https://github.com/Alejandro-BR)
* [Alvaro López Guerrero](https://github.com/Alvalogue72)
* [Andrei Munteanu Popa](https://github.com/andu8705)

Máster de FP en Inteligencia Artifical y Big Data - CPIFP Alan Turing - `Curso 2025/2026`

### 1. Importacion de librerias
En esta celda importamos las herramientas necesarias. Usamos `undetected_chromedriver` como navegador principal para evadir la deteccion, `selenium` para la interaccion con elementos web, `pandas` para la estructura de datos, y `io.StringIO` para convertir el HTML en un formato legible para Pandas sin generar advertencias de depreciación.

In [ ]:

import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import json
from io import StringIO
import time
import random
from bs4 import BeautifulSoup

### 2. Definicion de categorias y configuracion de limites
Definimos un diccionario con las URLs de PCPartPicker que queremos analizar:

1. Las claves del diccionario serviran para nombrar los DataFrames posteriormente. Se pueden añadir más categorias siguiendo este mismo formato.

2. Se ha utilizado la version española (.es), pero se puede quitar si es que el usuario prefiere los precios en dolares americanos.

Ademas, definimos el limite maximo de paginas que queremos recorrer y la ruta donde se guardara nuestro perfil de Chrome para evitar los bloqueos de Cloudflare.

In [ ]:
# Usa float('inf') para que recolecte paginas de forma ilimitada
MAX_PAGES = float('inf')

CARPETA_PERFIL = os.path.join(os.path.expanduser("~"), "pcpartpicker_chrome_profile")

categories = {
    'cpu': 'https://es.pcpartpicker.com/products/cpu/',
    'cpu_cooler': 'https://es.pcpartpicker.com/products/cpu-cooler/',
    'gpu': 'https://es.pcpartpicker.com/products/video-card/',
    'ram': 'https://es.pcpartpicker.com/products/memory/',
    'motherboard': 'https://es.pcpartpicker.com/products/motherboard/',
    'storage': 'https://es.pcpartpicker.com/products/internal-hard-drive/',
    'cases': 'https://es.pcpartpicker.com/products/case/',
    'psu': 'https://es.pcpartpicker.com/products/power-supply/',
    'os': 'https://es.pcpartpicker.com/products/os/',
    'monitor': 'https://es.pcpartpicker.com/products/monitor/',
}

### 3. Configuracion del Driver
Esta funcion inicializa el navegador:

1. Configuramos el driver de Chrome para que se comporte como un usuario real.

2. Se establece un tiempo de carga de pagina implicito para dar margen a la conexion.

In [ ]:
def setup_driver():
    if not os.path.exists(CARPETA_PERFIL):
        os.makedirs(CARPETA_PERFIL)

    options = uc.ChromeOptions()
    options.add_argument('--no-first-run')
    options.add_argument('--no-service-autorun')
    options.add_argument('--password-store=basic')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    
    # Inyección de Perfil Persistente
    options.add_argument(f"--user-data-dir={CARPETA_PERFIL}")
    
    driver = uc.Chrome(options=options, version_main=None)
    driver.implicitly_wait(10)
    return driver

# Detecta si Cloudflare pide verificacion manual y pausa el script.
def comprobar_cloudflare(driver):
    """Detecta si Cloudflare pide verificación manual o si hay un Rate Limit."""
    try:
        titulo = driver.title.lower()
        # Detección de Captchas de Cloudflare
        if "just a moment" in titulo or "cloudflare" in titulo or "attention required" in titulo:
            print("\n[!] CLOUDFLARE DETECTADO")
            try:
                WebDriverWait(driver, 120).until_not(EC.title_contains("Just a moment"))
                time.sleep(2)
            except TimeoutException:
                pass
                
        # Deteccion de "Soft-Bans" por demasiadas peticiones (Error 429)
        elif "429" in titulo or "too many requests" in titulo:
            print("\n[!] RATE LIMIT: Espera 60 segundos a que el script se reanude...")
            time.sleep(60)
            driver.refresh() # Recargamos tras la pausa
            time.sleep(5)
            
    except Exception:
        raise # Si se cierra la ventana durante el chequeo, pasamos el error arriba.

### 4. Scraping
Esta es la funcion principal. Su logica es la siguiente:
1. Navega a la URL.

2. Espera a que la tabla de productos (`#category_content`) sea visible.

3. Extrae el HTML de la tabla.

4. Utiliza `StringIO` para envolver el texto HTML y pasarlo a `pd.read_html`, que convierte automaticamente la tabla HTML en un DataFrame.

5. Usamos `BeautifulSoup` en el HTML extraido para buscar especificamente las etiquetas `<img>` dentro de las filas (`tr`).

6. Extraemos el atributo `src` de la imagen e insertamos la lista de URLs como una columna nueva en el DataFrame.

7. Realiza una limpieza basica: elimina columnas vacias (comunes en PCPartPicker por los botones de "Add") y filas nulas.

**UPDATE:** Esta funcion ha sido optimizada. En lugar de encargarse de hacer clics en el boton "Next", ahora recibe una URL especifica de una pagina (`#page={num}`) y devuelve unicamente el DataFrame con los datos de esa pagina concreta. Esto nos permite controlar la paginacion de forma externa.

In [ ]:
def scrape_single_page(driver, url):
    driver.get(url)
    comprobar_cloudflare(driver)
    
    try:
        # Esperamos a que la tabla sea visible
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "category_content"))
        )
        time.sleep(random.uniform(2.0, 4.0))
        
        table_element = driver.find_element(By.CSS_SELECTOR, "table")
        table_html = table_element.get_attribute('outerHTML')
        
        # Transformamos el HTML crudo en un DataFrame con Pandas
        html_buffer = StringIO(table_html)
        df_page = pd.read_html(html_buffer)[0]
        
        # Extracción de URLs de imágenes usando BeautifulSoup
        soup = BeautifulSoup(table_html, 'html.parser')  
        rows = soup.find('tbody').find_all('tr')
        
        image_urls = []
        for row in rows:
            img_tag = row.find('img')
            if img_tag and img_tag.get('src'):
                src = img_tag.get('src')
                if src.startswith('//'):
                    src = 'https:' + src
                image_urls.append(src)
            else:
                image_urls.append(None)
        
        # Alineamos las imagenes con el DataFrame
        if len(df_page) == len(image_urls):
            df_page['image_url'] = image_urls
        else:
            while len(image_urls) < len(df_page):
                image_urls.append(None)
            df_page['image_url'] = image_urls[:len(df_page)]

        # Limpieza básica
        if not df_page.empty:
            df_page = df_page.dropna(axis=1, how='all')
            
        return df_page

    except Exception as e:
        return pd.DataFrame()

### 5. Main Loop
En esta celda se orquesta todo el proceso:
1. Iteramos pagina por página y dentro, iteramos por cada componente de la lista

2. Si Pandas devuelve una tabla vacía, significa que ese componente en particular ya no tiene mas paginas. Si esto pasa, se elimina esa categoría de la lista para no perder tiempo en las siguientes vueltas.

3. **Guardado:** Los datos se guardan linea por línea usando `orient='records', lines=True` de Pandas directamente en archivos `.jsonl` independientes.

In [ ]:
if not os.path.exists('pcpartpicker'):
    os.makedirs('pcpartpicker')

driver = setup_driver()
categorias_activas = list(categories.keys())
page = 1

try:
    # El bucle se mantiene mientras no alcancemos MAX_PAGES y queden categorias con datos
    while page <= MAX_PAGES and categorias_activas:
        print(f"\nScraping pagina {page} de categoria...")
        
        # Iteramos sobre una copia de la lista para poder eliminar elementos de forma segura
        for cat_name in categorias_activas.copy():
            print(f"\nProcesando catalogo: {cat_name}")
            
            # Construimos la URL dinámica apoyandonos en el sistema de "anclaje" de PCPartPicker
            url_dinamica = f"{categories[cat_name]}#page={page}"
            
            df_page = scrape_single_page(driver, url_dinamica)
            
            # Si Pandas no encuentra tabla, el catálogo ha llegado a su fin
            if df_page is None or df_page.empty:
                print(f"\nFin de datos para '{cat_name}'.")
                categorias_activas.remove(cat_name)
            else:
                # Guardado inmediato en formato JSONL
                file_path = f"pcpartpicker/{cat_name}_pcpartpicker.jsonl"
                
                # Convertimos el DataFrame a texto JSONL plano
                jsonl_str = df_page.to_json(orient='records', lines=True, force_ascii=False)
                
                with open(file_path, 'a', encoding='utf-8') as f:
                    f.write(jsonl_str)
                    if not jsonl_str.endswith('\n'):
                        f.write('\n')
                        
                print(f"Datos agregados!")
        
        page += 1

    print("\nScript completado, un saludo socio!")

except KeyboardInterrupt:
    print("\n\n[!] AVISO: El script ha sido detenido manualmente por el usuario.")
    print("[!] Los datos procesados se han guardado en el archivo JSONL.")

finally:
    if driver:
        try:
            driver.quit()
        except:
            pass